In [41]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
import numpy
print(numpy.__version__)

1.19.4


In [43]:
import sklearn
print(sklearn.__version__)

0.23.2


In [44]:
#import sys
#!{sys.executable} -m pip uninstall numpy -y

In [45]:
#import sys
#!{sys.executable} -m pip install numpy==1.24.2

In [46]:
import sys
!{sys.executable} -m pip install dask-ml==1.9.0

In [47]:
import sys
!{sys.executable} -m pip install scikit-learn-intelex

In [48]:
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [49]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os

In [50]:
df_pd = pd.read_csv('https://gitlab.com/mirsakhawathossain/exodata/-/raw/main/dataset/exodata.csv').sort_index(axis=0)

In [51]:
columns_1 = df_pd.columns[df_pd.mean()==1]
columns_2 = df_pd.columns[df_pd.mean()==0]
df_pd = df_pd.drop(columns=columns_1)
df_pd = df_pd.drop(columns=columns_2)

In [52]:
from dask.distributed import Client
import joblib

client = Client(n_workers=os.cpu_count())
client

Client Scheduler: tcp://127.0.0.1:44444 Dashboard: http://127.0.0.1:42066/status,Cluster Workers: 16 Cores: 16 Memory: 107.37 GB


In [53]:
#df_dd = dd.from_pandas(df_pd, npartitions=100)
#df_dd = df_dd.persist()

In [54]:
#df_dd

In [55]:
X = df_pd.drop(['exoplanet'],axis=1)

In [56]:
#X

In [57]:
y = df_pd[['exoplanet']]

In [58]:
X = dd.from_pandas(X, npartitions=100)
X = X.persist()
y = dd.from_pandas(y, npartitions=100)
y = y.persist()

In [59]:
from dask_ml.model_selection import train_test_split

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, shuffle=True)

In [61]:
print(X_train.shape)
print(X_test.shape)

(Delayed('int-e9f8645e-cdd9-4f31-90a4-87399e3b5b63'), 741)
(Delayed('int-a0848581-9ddb-42dc-8eec-102a1e1a8d16'), 741)


In [62]:
print(y_test.shape)
print(y_test.shape)

(Delayed('int-c06b069f-c3ea-4d3c-8763-72f107e712db'), 1)
(Delayed('int-75afa7b3-114b-4754-9739-95c69e3d87ef'), 1)


In [63]:
#from sklearn.preprocessing import QuantileTransformer
#qt = QuantileTransformer()

In [64]:
from dask_ml.preprocessing import QuantileTransformer
qt = QuantileTransformer()

In [65]:
X_train = qt.fit_transform(X_train)
X_test = qt.transform(X_test)

In [66]:
#import dask.array as da


In [67]:
#X_train = da.from_array(X_train)  
#X_test = da.from_array(X_test)
#y_train = y_train.to_dask_array(lengths=True)
#y_test = y_test.to_dask_array(lengths=True) 

In [68]:
#X_train = X_train.rechunk(chunks='auto')
#X_test = X_test.rechunk(chunks='auto')
#y_train = y_train.rechunk(chunks='auto')
#y_test = y_test.rechunk(chunks='auto')

In [33]:
X_test.compute_chunk_sizes()

dask.array<values, shape=(30166, 1), dtype=int64, chunksize=(334, 1), chunktype=numpy.ndarray>

In [69]:
# convert the dataframe to a dask array
import dask.array as da
y_train = y_train.to_dask_array(lengths=True)
y_test = y_test.to_dask_array(lengths=True)

In [28]:
#from sklearn.model_selection import RepeatedKFold
#cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=41)

In [70]:
import dask_ml.model_selection as dcv
from sklearn.ensemble import RandomForestClassifier

In [71]:
clf = RandomForestClassifier(random_state=43,n_jobs = -1)

In [72]:
params = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
          'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
          'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
          'max_features': ['sqrt', 'log2', None],
          'criterion': ['gini', 'entropy'],
          'n_estimators': [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]}

In [73]:
hyp_se = dcv.HyperbandSearchCV(estimator=clf,parameters=params,max_iter=10,random_state=45,scoring='accuracy', verbose=True)

In [74]:
hyp_se.fit(X_train, y_train)

TypeError: Cannot operate on Dask array with unknown chunk sizes. Use the following the compute chunk sizes:

   >>> X.compute_chunk_sizes()  # if Dask.Array
   >>> ddf.to_dask_array(lengths=True)  # if Dask.Dataframe

In [ ]:
# summarize result
print('Best Score: %s' %hyp_se.best_score_)
print('Best Hyperparameters: %s' %hyp_se.best_params_)
print('Best Estimator: %s' %hyp_se.best_estimator_)

In [ ]:
print("Test set accuracy score:", hyp_se.score(X_test, y_test))